In [71]:
pip install sqlalchemy

In [72]:
#Imports
import pandas as pd
from sqlalchemy import create_engine, inspect
import datetime

In [73]:
#create engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [74]:
# Table Names
table_names = inspector.get_table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [75]:
# Create Dataframe
revenue = pd.read_sql_query("SELECT name, "
                                 "SUM(CASE WHEN memid = 0 THEN slots * guestcost ELSE slots * membercost END) AS revenue "
                            "FROM Bookings "
                             "LEFT JOIN Facilities "
                             "USING(facid) "
                             "GROUP BY name "
                             "HAVING revenue < 1000;", engine)


In [76]:
revenue.head()

,name,revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [77]:
# Create Dataframe
report = pd.read_sql_query("SELECT "
                 "m1.firstname || ' ' || m1.surname AS member_name, "
                 "m2.firstname || ' ' || m2.surname AS recommended_by "
                 "FROM Members AS m1 "
                 "LEFT JOIN Members AS m2 ON m1.recommendedby = m2.memid "
                 "ORDER BY m1.surname, m1.firstname;", engine)

In [78]:
report.head()

,member_name,recommended_by
0,Florence Bader,Ponder Stibbons
1,Anne Baker,Ponder Stibbons
2,Timothy Baker,Jemima Farrell
3,Tim Boothe,Tim Rownam
4,Gerald Butters,Darren Smith


/* Q12: Find the facilities with their usage by member, but not guests */

In [79]:
usage = pd.read_sql_query("SELECT f.name AS facility_name, "
                            "m.firstname || ' ' || m.surname AS member_name, "
                            "COUNT(b.slots) AS usage_by_members "
                            "FROM Facilities AS f "
                            "INNER JOIN Bookings AS b ON f.facid = b.facid "
                            "INNER JOIN Members AS m ON b.memid = m.memid "
                            "WHERE b.memid != 0 "
                            "GROUP BY facility_name, member_name;", engine)

In [80]:
usage.head()

,facility_name,member_name,usage_by_members
0,Badminton Court,Anna Mackenzie,30
1,Badminton Court,Anne Baker,10
2,Badminton Court,Burton Tracy,2
3,Badminton Court,Charles Owen,6
4,Badminton Court,Darren Smith,132


/* Q13: Find the facilities usage by month, but not guests */

In [81]:
# Create Dataframe
usage_by_month = pd.read_sql_query("select f.name AS facility_name, "
                                   "strftime('%Y-%m', b.starttime) AS month, "
                                   "SUM(b.slots) AS usage_by_members "
                                   "From Facilities AS f "
                                   "Inner JOIN Bookings AS b ON f.facid = b.facid "
                                   "WHERE b.memid != 0 "
                                   "GROUP BY facility_name, month "
                                   "ORDER BY facility_name, month; ", engine)

In [82]:
usage_by_month.head()

,facility_name,month,usage_by_members
0,Badminton Court,2012-07,165
1,Badminton Court,2012-08,414
2,Badminton Court,2012-09,507
3,Massage Room 1,2012-07,166
4,Massage Room 1,2012-08,316
